# Stock data analysis

In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
from urllib.parse import urlencode
import datetime

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [5]:
class YahooFinanceWebsite:
    def __init__(self, stock_symbol):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.stock_symbol = stock_symbol.upper()

    def __build_url(self, params):
        base_url = f"https://finance.yahoo.com/quote/{self.stock_symbol}/history/"
        query_string = urlencode(params)
        return f"{base_url}?{query_string}"

    def get_stock_data(self):
        datetime_now = datetime.datetime.now()
        datetime_year_ago = datetime_now - datetime.timedelta(days=365)
        params = {"frequency": "1wk", "period1": datetime_year_ago.timestamp(), "period2": datetime_now.timestamp()}
        url = self.__build_url(params)
        response = requests.get(url, headers=headers)

        soup = BeautifulSoup(response.content, 'html.parser')
        
        title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()

        html_table_data = soup.find("table")

        return title, html_table_data

In [6]:
def build_stock_analysis_prompt(stock_symbol, title, stock_table_data):
    sys_prompt = r"""You are an assistant that analyzes the contents of HTML formated table that contains data on a specific stock.
    The HTML table contains the date, open price, close price, low and highs aggregated for every week over one year timeframe.
    Ignoring text, tags or html attributes that might be navigation related. 
    Respond in Markdown format"""
    
    user_prompt = f"The data provided below in the HTML table format for {stock_symbol} from the Yahoo Finances.\
    Make the explaination easy enough for a newbie to understand. \
    Analyze and Summarize the trends on this stock:\n{stock_table_data}\n\n\
    Also, calculate the total returns in percentage one could have expected over this period."
    
    return [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt}
    ]

In [7]:
def analyze_stock_trends(stock_symbol):
    stock_data_page = YahooFinanceWebsite(stock_symbol)
    title, stock_table_data = stock_data_page.get_stock_data()
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = build_stock_analysis_prompt(stock_symbol, title, stock_table_data)
    )
    return response.choices[0].message.content

def display_analysis(stock_symbol):
    display(Markdown(analyze_stock_trends(stock_symbol)))

In [8]:
display_analysis("GOOG")

# Analysis of GOOG Stock Performance (1 Year Timeframe)

The data provided summarizes the weekly stock performance for Alphabet Inc. (GOOG), covering a year from February 2024 to February 2025. Let's break down the key elements of this data and what they signify.

## Key Terms
1. **Open Price**: The price at which the stock started trading for that week.
2. **High Price**: The highest price recorded during the week.
3. **Low Price**: The lowest price recorded during the week.
4. **Close Price**: The price at which the stock finished trading for that week.
5. **Adjusted Close**: The closing price adjusted for stock splits and dividends.
6. **Volume**: The total number of shares traded during the week.

## Stock Trends
### Overall Market Sentiment
- **Starting Price (Feb 2024)**: The stock started at approximately **143.45** on February 26, 2024.
- **Ending Price (Feb 2025)**: The stock price settled at **181.58** on February 21, 2025.

### Price Fluctuations
- There have been noticeable fluctuations throughout the year:
  - **Peak Prices**: The highest value was approximately **208.70** on February 3, 2025, which indicates significant upward momentum at the beginning of the year.
  - **Lowest Prices**: The lowest recorded price was **136.29** on March 4, 2024. 

### Volume Insights
- Trading volumes varied significantly, with some weeks showing very high trading activity:
  - The week of January 27, 2025, recorded a volume of approximately **84.86 million shares**, which indicates strong investor interest during that period.
  - The highest volume, **139.05 million shares**, occurred on February 3, 2025, coinciding with the week of the peak price.

### Dividend Payments
- There were dividend payments of **$0.20** on several occasions, indicating a consideration for income-seeking investors.

## Summary of Trends
- Over this period, the stock has shown an overall upward trend despite experiencing volatility. The stock price increased from **143.45** to **181.58**.
- The closing prices reflect a consistent effort to recover from lower prices observed earlier in the year. 
- The spike in price in early 2025 indicates bullish sentiments possibly due to positive news or fundamentals.
  
## Total Returns Calculation
To calculate the total return in percentage:

1. **Calculate the absolute return**:
   \[
   \text{Total Return} = \left(\frac{\text{Ending Price} - \text{Starting Price}}{\text{Starting Price}}\right) \times 100
   \]
   \[
   \text{Total Return} = \left(\frac{181.58 - 143.45}{143.45}\right) \times 100 \approx 26.60\%
   \]

### Conclusion
Investors looking into GOOG during this timeframe could have expected a total return of approximately **26.60%**, despite the volatility. This showcases a relative strength and growth potential for the stock, making it an interesting option for both growth and income-focused investors considering the dividends. 

Understanding these trends helps in making informed investment decisions. Always consider both market conditions and company performance before trading stocks.

In [ ]:
display_analysis("PFE")

In [ ]:
display_analysis("AAPL")